In [33]:
import pandas as pd
import numpy as np
import json
from pandas.api.types import CategoricalDtype
import re
from utils import *

In [17]:
mutations = pd.read_csv('/n/data1/hms/dbmi/farhat/Sanjana/MIC_data/WHO_resistance_variants_all.csv')

In [18]:
# Switching the abbreviations to the ones that I've been using, and removing any that I haven't been working with
drug_translation = {'AMI': 'amk', 'BDQ': 'bdq', 'CAP':'cap', 'CFZ':'cfz', 'DLM':'dlm', 'EMB':'emb', 'ETH':'eth', 'INH':'inh', 'KAN':'kan',
       'LZD':'lzd', 'PZA':'pza', 'RIF':'rif', 'STM':'str', 'LEV':'levo', 'MXF': 'moxi'}
mutations.drug = [drug_translation[original] for original in mutations.drug]
mutations = mutations.loc[[drug in ['amk', 'cap', 'emb', 'eth', 'inh', 'kan', 'rif', 'pza', 'str', 'levo'] for drug in mutations.drug],:]
mutations['searchable_variant'] = [str(i).replace('-','').upper() for i in list(mutations.variant)]

In [30]:
mutations[(mutations.genome_index == '3073868')]

,Unnamed: 0,drug,genome_index,confidence,gene,variant,searchable_variant


In [4]:
with open('//home//kin672//gentb-summer22//Isolating Relevant Files//7-18 Output//relevant_file_names_7-18.json') as f:
    relevant_files = json.load(f) # This is a dictionary with format {STRAIN ID: {Folder: , Resistance: , Lineage: }

In [5]:
def break_down(string):
    broken = {'original': string, 'gene': [], '1': [], '2': [], 'genome_index': [], 'change': []}
    temp = re.split('_|\\.', string)
    if not len(temp) >= 5:
        print(string)
    try:
        broken['gene'] = temp.pop(-1)
        broken['1'] = temp.pop(0)
        broken['2'] = temp.pop(0)
        broken['genome_index'] = temp.pop(0)
        broken['change'] = temp
    except:
        return(broken)

    return(broken)


In [6]:
final_db = pd.DataFrame({'isolate':[], 'drug':[], 'confidence':[], 'gene':[], 'genome_index':[], 'GenTB Mutation':[], 'WHO Variant':[]})

In [15]:
problems = {'The JSON is weird': [], 'This mutation is weird': {'Isolate':[], 'Mutation':[]}}
mutation_count = {'Isolate':[], 'Num_Mutations':[]}

prefix = '//n//groups//gentb_www//predictData//'

# Loop through every unique isolate
for strainID in relevant_files:
    value = relevant_files.get(strainID)
    json_path = prefix + value.get('Folder') + '//' + value.get('Resistance')
    
    # Parse json
    with open(json_path) as f:
        resistance = json.load(f) 
    
    # Get full list of resistance-related mutations for this isolate
    try:
        mut_list = [i for sublist in resistance[1].values() for sublist2 in sublist for i in sublist2 if i != None and i != 'Null']
        mut_list = mut_list + [i for sublist in resistance[2].values() for sublist2 in sublist for i in sublist2 if i != None and i != 'Null']
        mut_list = mut_list + [item for sublist in resistance[3].values() for item in sublist if item != None and item != 'Null']
        mut_list = list(pd.Series(mut_list).unique())
    except:
        try:
            mut_list = [i for sublist in resistance[1].values() for sublist2 in sublist for i in sublist2 if i != None and i != 'Null']
            mut_list = mut_list + [i for sublist in resistance[2].values() for sublist2 in sublist for i in sublist2 if i != None and i != 'Null']
        except:
            problems['The JSON is weird'].append(str(strainID))
            continue
    
    

In [16]:
mut_list

['SNP_P_4243221_C12T_promoter_embA.embB',
 'SNP_P_2289252_T11C_promoter_pncA',
 'SNP_P_4243222_C11A_promoter_embA.embB',
 'SNP_CN_4407809_C394A_D132Y_gid',
 'SNP_P_2289245_T4A_promoter_pncA',
 'SNP_P_4243225_C8T_promoter_embA.embB',
 'DEL_CF_4407640_d563A_188_gid',
 'DEL_CF_2288939_d303TCCGGTGTAG_101_pncA',
 'SNP_CN_4408138_T65C_Y22C_gid',
 'DEL_CF_2289069_d173A_58_pncA',
 'SNP_CN_4407748_A455G_L152S_gid',
 'DEL_CF_2288776_d466GCACCCTG_156_pncA',
 'SNP_P_4243217_C16T_promoter-embA-embB',
 'SNP_CN_4408162_G41A_P14L_gid',
 'DEL_CF_4326188_d1286CG_429_ethA',
 'A_promoter-embA-embB',
 '_14_gid',
 'SNP_CN_4326188_C1286T_R429Q_ethA',
 'SNP_P_4243221_C12T_promoter-embA-embB',
 'DEL_CF_4408163_d40GT_14_gid',
 '_429_ethA',
 'G_promoter-embA-embB',
 'SNP_CN_4408163_G40C_P14A_gid',
 '1285CT_R429R_ethA',
 'SNP_P_4243222_C11A_promoter-embA-embB',
 '_13G_gid',
 'C_428E_ethA',
 'SNP_CN_1674476_T275G_V92G_inhA',
 'SNP_CN_760309_T503G_V168G_rpoB',
 'T_promoter-pncA',
 'G_73_embB',
 'SNP_CN_4407538_C665

In [14]:
with open('//home//kin672//gentb-summer22//temp.json', 'w') as f1:
    json.dump(resistance, f1,  indent=4)

In [81]:
match_snp_name('DEL_CF_4408163_d40GT_14_gid')


{'mode': 'DEL',
 'syn': 'CF',
 'ntpos': '4408163',
 'codes': '40GT',
 'cpos': '40',
 'cref': 'GT',
 'noncode': '14',
 'gene': 'gid',
 'amino': None,
 'aref': None,
 'apos': None,
 'aver': None}

In [70]:
pd.concat([pd.DataFrame([match_snp_name('SNP_CN_1674480_T279G_H93Q_inhA')]), pd.DataFrame([match_snp_name('SNP_CN_1674480_T279G_H93Q_inhA')]), pd.DataFrame([match_snp_name('SNP_CN_1674480_T279G_H93Q_inhA')])])

,syn,ntpos,coding,cref,cpos,cver,amino,aref,apos,aver,noncode,gene,rgene
0,CN,1674480,T279G,T,279,G,H93Q,H,93,Q,None,inhA,None
0,CN,1674480,T279G,T,279,G,H93Q,H,93,Q,None,inhA,None
0,CN,1674480,T279G,T,279,G,H93Q,H,93,Q,None,inhA,None


In [ ]:
# Record the total number of mutations
    mutation_count['Isolate'].append(strainID)
    mutation_count['Num_Mutations'].append(len(mut_list))
    
    # Don't even attempt to parse if the list is empty
    if len(mut_list) == 0:
        continue
    
    # Match the mutation with the corresponding mutation(s) in the WHO database
    for mut in mut_list:
        x = break_down(mut)
        if x == 'Bad Breakdown' or [] in x.values():
            problems['This mutation is weird']['Isolate'].append(str(strainID))
            problems['This mutation is weird']['Mutation'].append(str(mut))
            continue
            
        candidates = mutations.loc[mutations.genome_index == x['genome_index'],:]
        
        # Make sure the gene aligns
        candidates = candidates.loc[[gene in x['gene'] for gene in candidates.gene], :]
        
        # Isolate the candidates who have one of the amino acid swaps indicated
        candidates = candidates.loc[[any(code in searchable_variant for code in x['change']) for searchable_variant in candidates.searchable_variant], :]  
        
        # Move on if there are no relevant mutations
        if len(candidates) == 0:
            continue
            
        temp = candidates.loc[:,['drug', 'genome_index', 'confidence', 'gene']]
        temp['WHO Variant'] = candidates.variant
        temp['isolate'] = strainID
        temp['GenTB Mutation'] = x['original']
        
        final_db = pd.concat([final_db, temp], ignore_index = True)
            

In [190]:
final_db.to_csv('/home/kin672/gentb-summer22/Mutations/all_mutations.csv', index = False)
pd.DataFrame(problems['The JSON is weird']).to_csv('/home/kin672/gentb-summer22/Mutations/Weird_JSONS.csv', index = False)
pd.DataFrame(problems['This mutation is weird']).to_csv('/home/kin672/gentb-summer22/Mutations/Weird_Mutations.csv', index = False)
pd.DataFrame(mutation_count).to_csv('/home/kin672/gentb-summer22/Mutations/Mutation_Count.csv', index = False)

SyntaxError: invalid syntax (596733944.py, line 1)

In [83]:
db = pd.read_csv('/home/kin672/gentb-summer22/Mutations/Mutation_Info_Martin.csv')
db = db.loc[:, ['Isolate',
             'Full Mutation',
             'mode',
             'codes',
             'syn',
             'ntpos',
             'coding',
             'cref',
             'cpos',
             'cver',
             'amino',
             'aref',
             'apos',
             'aver',
             'noncode',
             'gene',
             'rgene',
                ]
]

In [92]:
db[[not pd.isna(x) for x in db['mode']]]['mode'].unique()

array(['DEL', 'INS'], dtype=object)

In [118]:
db.loc[[pd.isna(x) for x in db['mode']], ['mode', 'codes']].drop_duplicates()

,mode,codes
0,NaN,NaN


In [127]:
db.loc[:, 'gene'].unique()

array(['katG', 'rpoB', 'embB', ..., '65GL_pncA', '119H_rpsA', '6V_eis'],
      dtype=object)